In [11]:
import matplotlib.pyplot as plt

from models import TransformerModelLooped, TransformerModelLoopedLastNTokens
from curriculum import CurriculumSimple
from train import train_without_config, validate_model

n_dims = 10

In [2]:
model_loop_b5 = TransformerModelLoopedLastNTokens(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
    n=10
).cuda()

cirriculum_b5 = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [5000, 5, 0])

metrics_l1_b5_d10 = train_without_config(model_loop_b5, cirriculum_b5, model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

In [3]:
model_loop_b10 = TransformerModelLooped(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
).cuda()

cirriculum_b10 = CurriculumSimple(n_dims, 31, 10, [5000, n_dims, 0], [5000, 31, 0], [5000, 10, 0])

metrics_l1_b10_d10 = train_without_config(model_loop_b10, cirriculum_b10, model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

In [4]:
model_loop_b20 = TransformerModelLooped(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
).cuda()

cirriculum_b20 = CurriculumSimple(n_dims, 31, 20, [5000, n_dims, 0], [5000, 31, 0], [5000, 20, 0])

metrics_l1_b20_d10 = train_without_config(model_loop_b20, cirriculum_b20, model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

In [12]:
def calculate_by_loop(model):
    val_by_loop = []
    loop_steps = []
    for i in range(1, 60):
        val_loss = validate_model(model, n_dims_truncated=10, n_loops=i, model_n_dims=10, n_points=21, family="gpt2_loop")
        val_by_loop.append(val_loss / n_dims)
        loop_steps.append(i)
    return loop_steps, val_by_loop 
steps_b5, values_b5 = calculate_by_loop(model_loop_b5)
steps_b10, values_b10 = calculate_by_loop(model_loop_b10)
steps_b20, values_b20 = calculate_by_loop(model_loop_b20)

In [14]:
plt.plot(steps_b5, values_b5)
plt.plot(steps_b10, values_b10)
plt.plot(steps_b20, values_b20)
plt.legend(["b=5", "b=10", "b=20"])
plt.savefig('../images/check_for_convergence_properties.png')